# Mining testimonial fragments of the Holocaust

**Experience domain:** stand

### Load the necessary libraries

In [45]:
import sys; sys.path.insert(0, '..')
import itertools

In [46]:
import get_topic_model_concordance as topic_concordancer
from utils import blacklab, db, text
mongo = db.get_db()

In [47]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random

### Helper functions

In [48]:
def create_contextual_query(lemmas,context_length=50):
    permutations = itertools.permutations(lemmas,len(lemmas))
    final_result = []
    for element in list(permutations):
        temp_result = []
        for el in element:
            temp_result.append('[lemma="'+el+'"]')
        temp_result = '('+('[]{0,'+str(context_length)+'}').join(temp_result)+')'
        final_result.append(temp_result)
    final_result = '|'.join(final_result)
    return final_result
        
        
            

In [49]:
from utils import blacklab, db, text
import requests
import json
def find_sentence_id(label):
    props = {'annotators': 'tokenize'}

    # set the encoding of the annotator
    requests.encoding = 'utf-8'
    # make a request
    r = requests.post('http://localhost:9000/', params={'properties':
                      json.dumps(props)},
                      data=label.encode('utf-8'))
    result = json.loads(r.text, encoding='utf-8')
    query = []
    for i, token in enumerate(result['tokens']):

        if ('...'in token['word'] and ((i == 0) or
           i == len(result['tokens']) - 1)):
            continue
        elif ('...'in token['word']):
            query.append('[]{0,50}')
        elif ('-'in token['word']):
            query.append('[]{0,3}')
        elif ("n't"in token['word']):
            query.append('[]')
        elif ("'re"in token['word']):
            query.append('[]')
        elif ("?"in token['word']):
            query.append('[]')
        elif ("."in token['word']):
            query.append('[]')
        elif ("'s"in token['word']):
            query.append('[]')
        elif (","in token['word']):
            query.append('[]')
        else:
            query.append('["' + token['word'] + '"]')

    query = ' '.join(query)
    try:
        sentence = blacklab.search_blacklab(query, window=0,
                                            lemma=False,
                                            include_match=True)
        token_end = sentence[0]['token_end']
        token_start = sentence[0]['token_start']
        print (sentence[0])
        mongo = db.get_db()
        results = mongo.tokens.find({'testimony_id':
                                    sentence[0]['testimony_id']},
                                    {'_id': 0})
        tokens = list(results)[0]['tokens']
        sentenceStart = tokens[token_start]['sentence_index']
        sentenceEnd = tokens[token_end]['sentence_index']
        originalsentence = sentence[0]['complete_match']
        return (sentenceStart,sentenceEnd,sentence[0]['testimony_id'])
    except:
        print("The following query returned a null result")
        print(query)
        
            


In [50]:
def create_parent_node(label):
    """Generate a root node for a tree structure."""
    testimony_id = random.randint(1, 20)
    node = {}
    node['label'] = label
    fragment = {'label': label,
                'essay_id': random.randint(1, 20),
                'tree': get_node(testimony_id, node, is_parent=True)}
    fragment['tree']['label'] = label

    return fragment

In [51]:
def get_node(testimony_id, node, is_parent=False):
    """Generate a parent or leaf node for a tree structure."""
    if is_parent:
        return {
            'label': node['label'],
            'testimony_id': random.randint(1, 20),
            'media_index': random.randint(1, 20),
            'media_offset': random.randint(1, 20),
            'start_sentence_index': random.randint(1, 20),
            'end_sentence_index': random.randint(1, 20),
            'children': [], }
    else:
        return {'label': node['label'],
                'testimony_id': node['testimony_id'],
                'media_index': float(node['media_index']),
                'media_offset': float(node['media_offset']),
                'start_sentence_index': float(node['start_sentence_index']),
                'end_sentence_index': float(node['end_sentence_index']),
                'children': [], }

In [52]:
def check_if_main_node_exist(node):
    results = mongo.fragments.find({'label':node},{'_id': 0})
    if len(results[0])==0:
        return False
    else:
        return True

In [53]:
def add_main_node(label):
    mongo.fragments.insert(create_parent_node(label))

In [54]:
def delete_main_node(label):
    mongo.fragments.delete_one({'label':label})

In [55]:
def add_testimonial_fragments(fragments):
    if check_if_main_node_exist(fragments['main_node']):
        results = mongo.fragments.find({'label':fragments['main_node']},{'_id':0})[0]
        mid_nodes = [element['label'] for element in results['tree']['children']]
        if fragments['mid_node'] in mid_nodes:
            print ("mid node exists cannot be added")
        else:
            
            mid_node = get_node('r',{'label':fragments['mid_node']},is_parent=True)
            for fragment in fragments['fragments']:
                leaf = get_node(fragment['testimony_id'],fragment)
                mid_node['children'].append(leaf)
            results['tree']['children'].append(mid_node)
            mongo.fragments.replace_one({'label':fragments['main_node']},results)

### Add the main node

In [56]:
main_node = "stand"
#delete_main_node("numbness")
add_main_node(main_node)

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


### Set up the query

In [57]:
query = '[lemma="stand"]'

result = topic_concordancer.main(query,window=25,topicn=25)

### Print the key topics

for i,element in enumerate(result['topic_documents']):
    print (i)
    topic_words =  element['topic_words'][1]
    print (topic_words)
    print ('\n')

### Analyze documents

i=5

for text in result['topic_documents'][i]['texts'][0:25]:
    print (text['matched_text_words'])
    print ('\n')

## Testimonial fragments

### 1.  For hours

In [58]:
query = '[lemma="stand"] []{0,15} ["hours"]'

In [59]:
domain_term = "for hours"

In [60]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [61]:
fragment_1 = {}
fragment_1['original_sentence'] = "You can stand one hour, two hours, three hours, four hours, five hours."
fragment_1['label']="You can stand one hour, two hours, three hours, four hours, five hours."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22You%22%5D+%5B%22can%22%5D+%5B%22stand%22%5D+%5B%22one%22%5D+%5B%22hour%22%5D+%5B%5D+%5B%22two%22%5D+%5B%22hours%22%5D+%5B%5D+%5B%22three%22%5D+%5B%22hours%22%5D+%5B%5D+%5B%22four%22%5D+%5B%22hours%22%5D+%5B%5D+%5B%22five%22%5D+%5B%22hours%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'You can stand one hour , two hours , three hours , four hours , five hours . ', 'right': '', 'complete_match': 'You can stand one hour , two hours , three hours , four hours , five hours . ', 'testimony_id': 'HVT-149', 'shelfmark': ['Fortunoff HVT-149'], 'token_start': 12994, 'token_end': 13012}


In [62]:
fragment_2 = {}
fragment_2['original_sentence'] = "you stood for six hours, eight hours, 10 hours a day"
fragment_2['label']="(..)you stood for six hours, eight hours, 10 hours a day (..)"
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22you%22%5D+%5B%22stood%22%5D+%5B%22for%22%5D+%5B%22six%22%5D+%5B%22hours%22%5D+%5B%5D+%5B%22eight%22%5D+%5B%22hours%22%5D+%5B%5D+%5B%2210%22%5D+%5B%22hours%22%5D+%5B%22a%22%5D+%5B%22day%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'you stood for six hours , eight hours , 10 hours a day ', 'right': '', 'complete_match': 'you stood for six hours , eight hours , 10 hours a day ', 'testimony_id': 'irn35788', 'shelfmark': ['USHMM RG-50.030*0523'], 'token_start': 4622, 'token_end': 4635}


In [63]:
fragment_3 = {}
fragment_3['original_sentence'] = "And we were just standing there for hours."
fragment_3['label']="And we were just standing there for hours."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22just%22%5D+%5B%22standing%22%5D+%5B%22there%22%5D+%5B%22for%22%5D+%5B%22hours%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And we were just standing there for hours . ', 'right': '', 'complete_match': 'And we were just standing there for hours . ', 'testimony_id': 'HVT-125', 'shelfmark': ['Fortunoff HVT-125'], 'token_start': 10986, 'token_end': 10995}


In [64]:
fragment_4 = {}
fragment_4['original_sentence'] = "routine began of being awakened at 5:00 AM and pushed out into the cold, and standing there for hours until they counted."
fragment_4['label']= "(..)routine began of being awakened at 5:00 AM and pushed out into the cold, and standing there for hours until they counted."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22routine%22%5D+%5B%22began%22%5D+%5B%22of%22%5D+%5B%22being%22%5D+%5B%22awakened%22%5D+%5B%22at%22%5D+%5B%225%3A00%22%5D+%5B%22AM%22%5D+%5B%22and%22%5D+%5B%22pushed%22%5D+%5B%22out%22%5D+%5B%22into%22%5D+%5B%22the%22%5D+%5B%22cold%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22standing%22%5D+%5B%22there%22%5D+%5B%22for%22%5D+%5B%22hours%22%5D+%5B%22until%22%5D+%5B%22they%22%5D+%5B%22counted%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'routine began of being awakened at 5:00 AM and pushed out into the cold , and standing there for hours until they counted . ', 'right': '', 'complete_match': 'routine began of being awakened at 5:00 AM and pushed out into the cold , and standing there for hours until they counted . ', 'testimony_id': 'HVT-65', 'shelfmark': ['Fortunoff HVT-65'], 'token_start': 15464, 'token_end': 15488}


In [65]:
fragment_5 = {}
fragment_5['original_sentence'] = "You had to stand in a appell in the morning for 2 hours."
fragment_5['label']= "You had to stand in a appell in the morning for 2 hours."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22You%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%22stand%22%5D+%5B%22in%22%5D+%5B%22a%22%5D+%5B%22appell%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22morning%22%5D+%5B%22for%22%5D+%5B%222%22%5D+%5B%22hours%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'You had to stand in a appell in the morning for 2 hours . ', 'right': '', 'complete_match': 'You had to stand in a appell in the morning for 2 hours . ', 'testimony_id': 'irn504639', 'shelfmark': ['USHMM RG-50.030*0145'], 'token_start': 11441, 'token_end': 11455}


In [66]:
add_testimonial_fragments(fragments)

### 2.  Cold

In [67]:
lemmas = ["stand","cold"]

In [68]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="stand"][]{0,25}[lemma="cold"])|([lemma="cold"][]{0,25}[lemma="stand"])


In [69]:
domain_term = "in cold"

In [70]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [71]:
fragment_1 = {}
fragment_1['original_sentence'] = "And he asked she should sing again, and the whole block, the whole group was standing on the floor. It was cold, it was already October, November."
fragment_1['label']="(..) the whole block, the whole group was standing on the floor. It was cold, it was already October, November."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22he%22%5D+%5B%22asked%22%5D+%5B%22she%22%5D+%5B%22should%22%5D+%5B%22sing%22%5D+%5B%22again%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22the%22%5D+%5B%22whole%22%5D+%5B%22block%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22whole%22%5D+%5B%22group%22%5D+%5B%22was%22%5D+%5B%22standing%22%5D+%5B%22on%22%5D+%5B%22the%22%5D+%5B%22floor%22%5D+%5B%5D+%5B%22It%22%5D+%5B%22was%22%5D+%5B%22cold%22%5D+%5B%5D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22already%22%5D+%5B%22October%22%5D+%5B%5D+%5B%22November%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And he asked she should sing again , and the whole block , the whole group was standing on the floor . It was cold , it was already October , November . ', 'right': '', 'complete_match': 'And he asked she should sing again , and the whole block , the whole group was standing on the floor . It was cold , it was already October , November 

In [72]:
fragment_2 = {}
fragment_2['original_sentence'] = "It was awfully cold, very cold and we stood probably until two or three o'clock in the morning"
fragment_2['label']="It was awfully cold, very cold and we stood probably until two or three o'clock in the morning (..)"
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22It%22%5D+%5B%22was%22%5D+%5B%22awfully%22%5D+%5B%22cold%22%5D+%5B%5D+%5B%22very%22%5D+%5B%22cold%22%5D+%5B%22and%22%5D+%5B%22we%22%5D+%5B%22stood%22%5D+%5B%22probably%22%5D+%5B%22until%22%5D+%5B%22two%22%5D+%5B%22or%22%5D+%5B%22three%22%5D+%5B%22o%27clock%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22morning%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "It was awfully cold , very cold and we stood probably until two or three o'clock in the morning ", 'right': '', 'complete_match': "It was awfully cold , very cold and we stood probably until two or three o'clock in the morning ", 'testimony_id': 'irn504802', 'shelfmark': ['USHMM RG-50.030*0307'], 'token_start': 18631, 'token_end': 18650}


In [73]:
fragment_3 = {}
fragment_3['original_sentence'] = "stand on that same spot the entire day, uh, when it was really cold."
fragment_3['label']="(..) stand on that same spot the entire day, uh, when it was really cold."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22stand%22%5D+%5B%22on%22%5D+%5B%22that%22%5D+%5B%22same%22%5D+%5B%22spot%22%5D+%5B%22the%22%5D+%5B%22entire%22%5D+%5B%22day%22%5D+%5B%5D+%5B%22uh%22%5D+%5B%5D+%5B%22when%22%5D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22really%22%5D+%5B%22cold%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'stand on that same spot the entire day , uh , when it was really cold . ', 'right': '', 'complete_match': 'stand on that same spot the entire day , uh , when it was really cold . ', 'testimony_id': 'HVT-137', 'shelfmark': ['Fortunoff HVT-137'], 'token_start': 11321, 'token_end': 11338}


In [74]:
fragment_4 = {}
fragment_4['original_sentence'] = "Then I started to cough because it was very cold and humid in Auschwitz 2:00 in the morning when you had to go out and stand the roll calls."
fragment_4['label']= "(..)I started to cough because it was very cold and humid (..) in the morning when you had to go out and stand the roll calls."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Then%22%5D+%5B%22I%22%5D+%5B%22started%22%5D+%5B%22to%22%5D+%5B%22cough%22%5D+%5B%22because%22%5D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22very%22%5D+%5B%22cold%22%5D+%5B%22and%22%5D+%5B%22humid%22%5D+%5B%22in%22%5D+%5B%22Auschwitz%22%5D+%5B%222%3A00%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22morning%22%5D+%5B%22when%22%5D+%5B%22you%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%22go%22%5D+%5B%22out%22%5D+%5B%22and%22%5D+%5B%22stand%22%5D+%5B%22the%22%5D+%5B%22roll%22%5D+%5B%22calls%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Then I started to cough because it was very cold and humid in Auschwitz 2:00 in the morning when you had to go out and stand the roll calls . ', 'right': '', 'complete_match': 'Then I started to cough because it was very cold and humid in Auschwitz 2:00 in the morning when you had to go out and stand the roll calls . ', 'testimony_id': 'HVT-43', '

In [75]:
fragment_5 = {}
fragment_5['original_sentence'] = "We stood in line half naked in the cold and splashed our faces."
fragment_5['label']= "We stood in line half naked in the cold and splashed our faces."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22stood%22%5D+%5B%22in%22%5D+%5B%22line%22%5D+%5B%22half%22%5D+%5B%22naked%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22cold%22%5D+%5B%22and%22%5D+%5B%22splashed%22%5D+%5B%22our%22%5D+%5B%22faces%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'We stood in line half naked in the cold and splashed our faces . ', 'right': '', 'complete_match': 'We stood in line half naked in the cold and splashed our faces . ', 'testimony_id': 'irn504816', 'shelfmark': ['USHMM RG-50.030*0322'], 'token_start': 19924, 'token_end': 19938}


In [76]:
add_testimonial_fragments(fragments)

### 3.  Hot

In [77]:
lemmas = ["stand","hot"]

In [78]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="stand"][]{0,25}[lemma="hot"])|([lemma="hot"][]{0,25}[lemma="stand"])


In [79]:
domain_term = "in the heat"

In [80]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [81]:
fragment_1 = {}
fragment_1['original_sentence'] = "at least 15,000 girls had to stand in the heat, five deep, and stand for hours just to count us twice a day."
fragment_1['label']="(..)at least 15,000 girls had to stand in the heat,(..) stand for hours just to count us twice a day. "
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22at%22%5D+%5B%22least%22%5D+%5B%5D+%5B%22girls%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%22stand%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22heat%22%5D+%5B%5D+%5B%22five%22%5D+%5B%22deep%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22stand%22%5D+%5B%22for%22%5D+%5B%22hours%22%5D+%5B%22just%22%5D+%5B%22to%22%5D+%5B%22count%22%5D+%5B%22us%22%5D+%5B%22twice%22%5D+%5B%22a%22%5D+%5B%22day%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'at least 15,000 girls had to stand in the heat , five deep , and stand for hours just to count us twice a day . ', 'right': '', 'complete_match': 'at least 15,000 girls had to stand in the heat , five deep , and stand for hours just to count us twice a day . ', 'testimony_id': 'HVT-102', 'shelfmark': ['Fortunoff HVT-102'], 'token_start': 6533, 'token_end': 6559}


In [82]:
fragment_2 = {}
fragment_2['original_sentence'] = "The whole day we stood in the hot sun"
fragment_2['label']="The whole day we stood in the hot sun (..)."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22The%22%5D+%5B%22whole%22%5D+%5B%22day%22%5D+%5B%22we%22%5D+%5B%22stood%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22hot%22%5D+%5B%22sun%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'The whole day we stood in the hot sun ', 'right': '', 'complete_match': 'The whole day we stood in the hot sun ', 'testimony_id': 'irn508630', 'shelfmark': ['USHMM RG-50.462*0009'], 'token_start': 4561, 'token_end': 4570}


In [83]:
fragment_3 = {}
fragment_3['original_sentence'] = "And then they let us stand in the hot, boiling sun for hours"
fragment_3['label']="And then they let us stand in the hot, boiling sun for hours (..)."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22then%22%5D+%5B%22they%22%5D+%5B%22let%22%5D+%5B%22us%22%5D+%5B%22stand%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22hot%22%5D+%5B%5D+%5B%22boiling%22%5D+%5B%22sun%22%5D+%5B%22for%22%5D+%5B%22hours%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And then they let us stand in the hot , boiling sun for hours ', 'right': '', 'complete_match': 'And then they let us stand in the hot , boiling sun for hours ', 'testimony_id': 'usc_shoah_935', 'shelfmark': ['USC 935'], 'token_start': 33172, 'token_end': 33186}


In [84]:
fragment_4 = {}
fragment_4['original_sentence'] = " it was one of those real hot days; and we were standing for hours."
fragment_4['label']= "(..)it was one of those real hot days; and we were standing for hours."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22it%22%5D+%5B%22was%22%5D+%5B%22one%22%5D+%5B%22of%22%5D+%5B%22those%22%5D+%5B%22real%22%5D+%5B%22hot%22%5D+%5B%22days%22%5D+%5B%22%3B%22%5D+%5B%22and%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22standing%22%5D+%5B%22for%22%5D+%5B%22hours%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'it was one of those real hot days ; and we were standing for hours . ', 'right': '', 'complete_match': 'it was one of those real hot days ; and we were standing for hours . ', 'testimony_id': 'irn504725', 'shelfmark': ['USHMM RG-50.030*0236'], 'token_start': 7309, 'token_end': 7325}


In [85]:
add_testimonial_fragments(fragments)

### 4. Rain  

In [86]:
lemmas = ["stand","rain"]

In [87]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="stand"][]{0,25}[lemma="rain"])|([lemma="rain"][]{0,25}[lemma="stand"])


In [88]:
domain_term = "in the rain"

In [89]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [90]:
fragment_1 = {}
fragment_1['original_sentence'] = "And you were standing there in the rain for about a day and a night "
fragment_1['label']="And you were standing there in the rain for about a day and a night (..)"
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22you%22%5D+%5B%22were%22%5D+%5B%22standing%22%5D+%5B%22there%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22rain%22%5D+%5B%22for%22%5D+%5B%22about%22%5D+%5B%22a%22%5D+%5B%22day%22%5D+%5B%22and%22%5D+%5B%22a%22%5D+%5B%22night%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And you were standing there in the rain for about a day and a night ', 'right': '', 'complete_match': 'And you were standing there in the rain for about a day and a night ', 'testimony_id': 'irn504761', 'shelfmark': ['USHMM RG-50.030*0277'], 'token_start': 923, 'token_end': 938}


In [91]:
fragment_2 = {}
fragment_2['original_sentence'] = "They were standing all day in the rain and in the evening they took them away."
fragment_2['label']="They were standing all day in the rain and in the evening they took them away."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22were%22%5D+%5B%22standing%22%5D+%5B%22all%22%5D+%5B%22day%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22rain%22%5D+%5B%22and%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22evening%22%5D+%5B%22they%22%5D+%5B%22took%22%5D+%5B%22them%22%5D+%5B%22away%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'They were standing all day in the rain and in the evening they took them away . ', 'right': '', 'complete_match': 'They were standing all day in the rain and in the evening they took them away . ', 'testimony_id': 'irn515645', 'shelfmark': ['USHMM RG-50.462*0120'], 'token_start': 10155, 'token_end': 10172}


In [92]:
fragment_3 = {}
fragment_3['original_sentence'] = "It was a day like raining, drizzling, a Thursday afternoon from 1:00 on. There we were standing out in the open with the children."
fragment_3['label']="It was a day like raining, drizzling (..). There we were standing out in the open with the children."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22It%22%5D+%5B%22was%22%5D+%5B%22a%22%5D+%5B%22day%22%5D+%5B%22like%22%5D+%5B%22raining%22%5D+%5B%5D+%5B%22drizzling%22%5D+%5B%5D+%5B%22a%22%5D+%5B%22Thursday%22%5D+%5B%22afternoon%22%5D+%5B%22from%22%5D+%5B%221%3A00%22%5D+%5B%22on%22%5D+%5B%5D+%5B%22There%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22standing%22%5D+%5B%22out%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22open%22%5D+%5B%22with%22%5D+%5B%22the%22%5D+%5B%22children%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'It was a day like raining , drizzling , a Thursday afternoon from 1:00 on . There we were standing out in the open with the children . ', 'right': '', 'complete_match': 'It was a day like raining , drizzling , a Thursday afternoon from 1:00 on . There we were standing out in the open with the children . ', 'testimony_id': 'irn515645', 'shelfmark': ['USHMM RG-50.462*0120'], 'token_start': 12168, 'to

In [93]:
fragment_4 = {}
fragment_4['original_sentence'] = "And it was a terrible rain, and with snow mixed. And there we were standing, five of us"
fragment_4['label']= "And it was a terrible rain, and with snow mixed. And there we were standing, five of us (..)"
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22a%22%5D+%5B%22terrible%22%5D+%5B%22rain%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22with%22%5D+%5B%22snow%22%5D+%5B%22mixed%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22there%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22standing%22%5D+%5B%5D+%5B%22five%22%5D+%5B%22of%22%5D+%5B%22us%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And it was a terrible rain , and with snow mixed . And there we were standing , five of us ', 'right': '', 'complete_match': 'And it was a terrible rain , and with snow mixed . And there we were standing , five of us ', 'testimony_id': 'usc_shoah_27687', 'shelfmark': ['USC 27687'], 'token_start': 14827, 'token_end': 14848}


In [94]:
add_testimonial_fragments(fragments)

### 5. Sick  

In [95]:
lemmas = ["sick","stand"]

In [96]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="sick"][]{0,25}[lemma="stand"])|([lemma="stand"][]{0,25}[lemma="sick"])


In [97]:
domain_term = "the sick"

In [98]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [99]:
fragment_1 = {}
fragment_1['original_sentence'] = "They shot all the sick girls, those who could really not stand."
fragment_1['label']="They shot all the sick girls, those who could really not stand."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22shot%22%5D+%5B%22all%22%5D+%5B%22the%22%5D+%5B%22sick%22%5D+%5B%22girls%22%5D+%5B%5D+%5B%22those%22%5D+%5B%22who%22%5D+%5B%22could%22%5D+%5B%22really%22%5D+%5B%22not%22%5D+%5B%22stand%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'They shot all the sick girls , those who could really not stand . ', 'right': '', 'complete_match': 'They shot all the sick girls , those who could really not stand . ', 'testimony_id': 'HVT-99', 'shelfmark': ['Fortunoff HVT-99'], 'token_start': 7807, 'token_end': 7821}


In [100]:
fragment_2 = {}
fragment_2['original_sentence'] = "all those sick people they were standing outside, and had to go out, and they were standing outside"
fragment_2['label']="all those sick people they were standing outside, and had to go out, and they were standing outside"
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22all%22%5D+%5B%22those%22%5D+%5B%22sick%22%5D+%5B%22people%22%5D+%5B%22they%22%5D+%5B%22were%22%5D+%5B%22standing%22%5D+%5B%22outside%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%22go%22%5D+%5B%22out%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22they%22%5D+%5B%22were%22%5D+%5B%22standing%22%5D+%5B%22outside%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'all those sick people they were standing outside , and had to go out , and they were standing outside ', 'right': '', 'complete_match': 'all those sick people they were standing outside , and had to go out , and they were standing outside ', 'testimony_id': 'irn505564', 'shelfmark': ['USHMM RG-50.042*0010'], 'token_start': 4210, 'token_end': 4230}


In [101]:
fragment_3 = {}
fragment_3['original_sentence'] = "No matter how sick you were, you had to go and stand out on appell."
fragment_3['label']="No matter how sick you were, you had to go and stand out on appell."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22No%22%5D+%5B%22matter%22%5D+%5B%22how%22%5D+%5B%22sick%22%5D+%5B%22you%22%5D+%5B%22were%22%5D+%5B%5D+%5B%22you%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%22go%22%5D+%5B%22and%22%5D+%5B%22stand%22%5D+%5B%22out%22%5D+%5B%22on%22%5D+%5B%22appell%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'No matter how sick you were , you had to go and stand out on appell . ', 'right': '', 'complete_match': 'No matter how sick you were , you had to go and stand out on appell . ', 'testimony_id': 'usc_shoah_13623', 'shelfmark': ['USC 13623'], 'token_start': 19286, 'token_end': 19303}


In [102]:
fragment_4 = {}
fragment_4['original_sentence'] = "Some people got sick just standing there because their clothing wasn't enough."
fragment_4['label']= "Some people got sick just standing there because their clothing wasn't enough."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Some%22%5D+%5B%22people%22%5D+%5B%22got%22%5D+%5B%22sick%22%5D+%5B%22just%22%5D+%5B%22standing%22%5D+%5B%22there%22%5D+%5B%22because%22%5D+%5B%22their%22%5D+%5B%22clothing%22%5D+%5B%22was%22%5D+%5B%5D+%5B%22enough%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "Some people got sick just standing there because their clothing was n't enough . ", 'right': '', 'complete_match': "Some people got sick just standing there because their clothing was n't enough . ", 'testimony_id': 'usc_shoah_12150', 'shelfmark': ['USC 12150'], 'token_start': 13470, 'token_end': 13484}


In [103]:
add_testimonial_fragments(fragments)